Costruire un sottografo del grafo di Twitter: 
- tutti i follower di @KevinRoitero
 - per ogni follower: atributi, descizione, meetriche pubbliche, se l'account è protetto, salvare una unica serializzazione (un Json unico)
- Per ogni follower del profilo usare Tweepy per scaricare il numero di tweet pubblicati da quel profilo durante l'ultima settimana, aggiungere tale info al Json al punto 1 
- per ogni follower cha hanno almeno un follower e non sono protected scaricare 1000 followers (salvando le info al punto 1)
- NetworkX -> grafo diretto (nodi -> kevin roitero e i follower id = profilo utente e ciascun nodo ha come attributo username-descrizione-numero di follower) 

In [1]:
# setup librerie 
import os, tweepy, json
import networkx as nx 
import pandas as pd 
import numpy as np 
import random as rn
import networkx as nx
import matplotlib.pyplot as plt 
# setup credenziali e inizializzazione client 
api_key = "d42BqiRZxWT9fCO093YABDzSx"
api_secret_token = "5o48EqF4s6SYwpsfZfsu9nENb9xZHPVmAJ6mMA6b8agXTDSghB"
access_token = "1588237699193921536-nZqckxn97PWMLHm1tMmATKrymglyIK"
access_secret = "BLIVYj24rplYT3LgsXQ1lvOgw5NmqzLI6b6D41gse61ae"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAECgiwEAAAAASxOXKB3inXrh0zL7rlp1qwHhpE0%3DeABsdc15BSvFEQJVnsPOiHZjs7vUmM5hJ7rDtD0xqokABlpLEI"


In [3]:
# setup serializzazione 
# Serializzazione con JSON -> i dati scaricati possono essere tanti, sappiamo che possiamo scaricare un numero limitato di tweet 
# sarebbe meglio serializzare in locale 
data_folder = "data"
# salvataggio di dati in locale 
def serialize_json(folder, filename, data): 
    # se la cartella non esiste la creo 
    if not os.path.exists(folder): 
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as folder_1: 
        json.dump(data, folder_1, ensure_ascii=False, indent=4)
        folder_1.close()
    print(f"Data serialized to path: {folder}/{filename}")
# lettura file JSON da locale 
def read_json(path): 
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file: 
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data 
    else: 
        print(f"No data found at path: {path}")
        return{}
# serialize_json(data_folder, "tweet.json", tweet)

In [7]:
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True) # specifico il wait_on_rate -> per valutare i limiti dell'endpoint 
# specifico gli user e inizializzo l'holder dei followers 
users_to_find = ["KevinRoitero"]
all_followers = {}
# itero sugli user andando a ricercare l'utente per recuperare i dati 
for current_user in users_to_find:
    response = client.get_user(username = current_user)
    user_id = response.data.id 
    username = response.data.username
    all_followers[username] = []                            # richiedo l'username dei follower 

    print(f"Looking for {current_user} followers.")
    # utilizzo Paginator perchè non so a priori quanti follower ho 
    for current_user_followers in tweepy.Paginator(client.get_users_followers, id = user_id, user_fields = ["description","protected"],tweet_fields=["public_metrics"], max_results = 1000):
        this_user_followers = []
        for follower in current_user_followers.data:
            found_follower = {
                "id":follower["id"],
                "name":follower["name"],
                "username":follower["username"],
                "description":follower["description"],
                # manca protected 
                "public_metrics": follower["public_metrics"],
                "protected" : follower["protected"]
            } 
            this_user_followers.append(found_follower)
        all_followers[username] += this_user_followers
        print(f"Found {len(all_followers[username])} followers for {username}")
serialize_json(data_folder, "followers_example.json", all_followers)

Looking for KevinRoitero followers.
Found 132 followers for KevinRoitero
Data serialized to path: data/followers_example.json
